# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_pd=pd.read_csv('C:/Users/veyce/Downloads/bootcamp_lessons/nu-chi-data-pt-02-2020-u-c/Homework/06-Python-APIs/Instructions/output_data/weather_data.csv')
del weather_pd['Unnamed: 0']
weather_pd.head()

,City,Country,Lat,Long,Max Temp,Humidity,Wind Speed,Cloud Cover
0,Puerto El Triunfo,SV,13.28,-88.55,29.00,79,2.10,Clouds
1,Puerto El Triunfo,SV,13.28,-88.55,29.00,79,2.10,Clouds
2,Atuona,PF,-9.80,-139.03,27.49,75,8.76,Clear
3,Atuona,PF,-9.80,-139.03,27.49,75,8.76,Clear
4,Vaini,TO,-21.20,-175.20,27.00,74,3.60,Clouds


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations=weather_pd[["Lat","Long"]]
weight=weather_pd["Humidity"].astype(float)
max_humidity=weight.max()

In [5]:
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations,weights=weight,
                               dissipating=False,max_intensity=max_humidity,point_radius=1)
fig.add_layer(heat_layer)


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
fitted_cities=weather_pd.loc[(weather_pd['Max Temp']>70) & (weather_pd['Max Temp']<80) & 
                             (weather_pd['Cloud Cover']==0) & (weather_pd['Wind Speed']<10),:]
fitted_cities=fitted_cities.dropna(how='any')
fitted_cities.reset_index(inplace=True)
del fitted_cities["index"]
fitted_cities.head()

,City,Country,Lat,Long,Max Temp,Humidity,Wind Speed,Cloud Cover


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotels=[]
for i in range(len(fitted_cities)):
    lattitude=fitted_cities.loc[i]['Lat']
    longitude=fitted_cities.loc[i]['Long']
    params={
        "location":f'{lattitude},{longitude}',
        "radius":5000,
        'type':'hotel',
        "key":g_key
    }
    
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response=requests.get(base_url,params=params).json()
    try:
        hotels.append(response['results'][0]['name'])
    except:
        hotels.append("")
fitted_cities["Hotel Name"]=hotels

fitted_cities.head()

,City,Country,Lat,Long,Max Temp,Humidity,Wind Speed,Cloud Cover,Hotel Name


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in fitted_cities.iterrows()]
locations = fitted_cities[["Lat", "Long"]]

In [16]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))